## ============ Imports ============ 

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models

import utils
import vision_transformer as vits
from vision_transformer import DINOHead

## ============ Extract features for Few-Shot Task  ============ 

In [ ]:
def extract_feature_pipeline():
    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.Resize(256, interpolation=3),
        pth_transforms.CenterCrop(224),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    dataset_train = ReturnIndexDataset(os.path.join(data_path, "train"), transform=transform)
    #dataset_val = ReturnIndexDataset(os.path.join( data_path, "val"), transform=transform)
    dataset_val = ReturnIndexDataset(os.path.join( data_path, "test"), transform=transform)
    
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        sampler=sampler,
        batch_size= batch_size_per_gpu,
        num_workers= num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_val = torch.utils.data.DataLoader(
        dataset_val,
        batch_size= batch_size_per_gpu,
        num_workers= num_workers,
        pin_memory=True,
        drop_last=False,
    )
    print(f"Data loaded with {len(dataset_train)} train and {len(dataset_val)} val imgs.")

    # ============ building network ... ============
    if "vit" in  arch:
        model = vits.__dict__[ arch](patch_size= patch_size, num_classes=0)
        print(f"Model { arch} { patch_size}x{ patch_size} built.")
    elif "xcit" in  arch:
        model = torch.hub.load('facebookresearch/xcit',  arch, num_classes=0)
    elif  arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[ arch](num_classes=0)
    else:
        print(f"Architecture { arch} non supported")
        sys.exit(1)
    model.cuda()
    utils.load_pretrained_weights(model,
                                  pretrained_weights, 
                                  checkpoint_key,  
                                  arch,  
                                  patch_size)
    model.eval()

    # ============ extract features ... ============
    print("Extracting features for train set...")
    train_features = extract_features(model, data_loader_train,  use_cuda)
    print("Extracting features for val set...")
    test_features = extract_features(model, data_loader_val,  use_cuda)

    if utils.get_rank() == 0:
        train_features = nn.functional.normalize(train_features, dim=1, p=2)
        test_features = nn.functional.normalize(test_features, dim=1, p=2)

    train_labels = torch.tensor([s[-1] for s in dataset_train.samples]).long()
    test_labels = torch.tensor([s[-1] for s in dataset_val.samples]).long()
    # save features and labels
    if  dump_features and dist.get_rank() == 0:
        torch.save(train_features.cpu(), os.path.join( dump_features, "trainfeat.pth"))
        torch.save(test_features.cpu(), os.path.join( dump_features, "testfeat.pth"))
        torch.save(train_labels.cpu(), os.path.join( dump_features, "trainlabels.pth"))
        torch.save(test_labels.cpu(), os.path.join( dump_features, "testlabels.pth"))
    return train_features, test_features, train_labels, test_labels


@torch.no_grad()
def extract_features(model, data_loader, use_cuda=True, multiscale=False):
    metric_logger = utils.MetricLogger(delimiter="  ")
    features = None
    for samples, index in metric_logger.log_every(data_loader, 10):
        samples = samples.cuda(non_blocking=True)
        index = index.cuda(non_blocking=True)
        if multiscale:
            feats = utils.multi_scale(samples, model)
        else:
            feats = model(samples).clone()

        # init storage feature matrix
        if dist.get_rank() == 0 and features is None:
            features = torch.zeros(len(data_loader.dataset), feats.shape[-1])
            if use_cuda:
                features = features.cuda(non_blocking=True)
            print(f"Storing features into tensor of shape {features.shape}")

        # get indexes from all processes
        y_all = torch.empty(dist.get_world_size(), index.size(0), dtype=index.dtype, device=index.device)
        y_l = list(y_all.unbind(0))
        y_all_reduce = torch.distributed.all_gather(y_l, index, async_op=True)
        y_all_reduce.wait()
        index_all = torch.cat(y_l)

        # share features between processes
        feats_all = torch.empty(
            dist.get_world_size(),
            feats.size(0),
            feats.size(1),
            dtype=feats.dtype,
            device=feats.device,
        )
        output_l = list(feats_all.unbind(0))
        output_all_reduce = torch.distributed.all_gather(output_l, feats, async_op=True)
        output_all_reduce.wait()

        # update storage feature matrix
        if dist.get_rank() == 0:
            if use_cuda:
                features.index_copy_(0, index_all, torch.cat(output_l))
            else:
                features.index_copy_(0, index_all.cpu(), torch.cat(output_l).cpu())
    return features


class ReturnIndexDataset(datasets.ImageFolder):
    def __getitem__(self, idx):
        img, lab = super(ReturnIndexDataset, self).__getitem__(idx)
        return img, idx

# ============ Setup the torch distributed ============ 

In [ ]:
def init_distributed_mode():
    # launched with torch.distributed.launch
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        rank = int(os.environ["RANK"])
        world_size = int(os.environ['WORLD_SIZE'])
        gpu = int(os.environ['LOCAL_RANK'])
    # launched with submitit on a slurm cluster
    elif 'SLURM_PROCID' in os.environ:
        rank = int(os.environ['SLURM_PROCID'])
        gpu = rank % torch.cuda.device_count()
    # launched naively with `python main_dino.py`
    # we manually add MASTER_ADDR and MASTER_PORT to env variables
    elif torch.cuda.is_available():
        print('Will run the code on one GPU.')
        rank, gpu, world_size = 0, 0, 1
        os.environ['MASTER_ADDR'] = '127.0.0.1'
        os.environ['MASTER_PORT'] = '29500'
    else:
        print('Does not support training without GPU.')
        sys.exit(1)

    dist.init_process_group(
        backend="nccl",
        init_method=dist_url,
        world_size=world_size,
        rank=rank,
    )

    torch.cuda.set_device(gpu)
    print('| distributed init (rank {}): {}'.format(
        rank, dist_url), flush=True)
    dist.barrier()

# ============ Setup the hyperparameters ============ 

In [ ]:
batch_size_per_gpu=128
temperature=0.07
pretrained_weights='../dino_ckpt/mini_imagenet_checkpoint.pth'
use_cuda=True
arch='vit_small'
patch_size=16
checkpoint_key="teacher"
dump_features=None
load_features=None
num_workers=10
dist_url="env://"
local_rank=0
data_path = '../data/'

init_distributed_mode()
print("git:\n  {}\n".format(utils.get_sha()))

cudnn.benchmark = True

# ============ Do the actual feature extraction here  ============ 

In [ ]:
if  load_features:
    train_features = torch.load(os.path.join( load_features, "trainfeat.pth"))
    test_features = torch.load(os.path.join( load_features, "testfeat.pth"))
    train_labels = torch.load(os.path.join( load_features, "trainlabels.pth"))
    test_labels = torch.load(os.path.join( load_features, "testlabels.pth"))
else:
    # need to extract features !
    train_features, test_features, train_labels, test_labels = extract_feature_pipeline()

## ============ Arrange for few-shot task  ============ 

In [ ]:
def unique(x, dim=None):
    """Unique elements of x and indices of those unique elements
    https://github.com/pytorch/pytorch/issues/36748#issuecomment-619514810

    e.g.

    unique(tensor([
        [1, 2, 3],
        [1, 2, 4],
        [1, 2, 3],
        [1, 2, 5]
    ]), dim=0)
    => (tensor([[1, 2, 3],
                [1, 2, 4],
                [1, 2, 5]]),
        tensor([0, 1, 3]))
    """
    unique, inverse = torch.unique(
        x, sorted=True, return_inverse=True, dim=dim)
    perm = torch.arange(inverse.size(0), dtype=inverse.dtype,
                        device=inverse.device)
    inverse, perm = inverse.flip([0]), perm.flip([0])
    return unique, inverse.new_empty(unique.size(0)).scatter_(0, inverse, perm)

train_features_cp, idx = unique(train_features, dim=0)
train_labels_cp = train_labels[idx.cpu().numpy()]

In [ ]:
train_dict = {}
train_dict.fromkeys(range(0, 64))

for i in range(64):
    idx = torch.where(train_labels_cp==i)[0]
    train_dict[i] = train_features_cp[idx.cpu().numpy()].cpu().numpy()

In [ ]:
import pickle
pickle.dump( train_dict, open( "../dino_features_data/dino_train_miniimagenet.p", "wb" ) )

In [ ]:
test_features_cp, idx = unique(test_features, dim=0)
test_labels_cp = test_labels[idx.cpu().numpy()]

In [ ]:
test_dict = {}
test_dict.fromkeys(range(0, 20))

for i in range(20):
    idx = torch.where(test_labels_cp==i)[0]
    test_dict[i+80] = test_features_cp[idx.cpu().numpy()].cpu().numpy()

In [ ]:
pickle.dump( test_dict, open( "../dino_features_data/dino_test_miniimagenet.p", "wb" ) )